In [173]:
import sys
sys.path.append('../')
import functions
exec(open('./functions.py').read())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chiso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\chiso\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chiso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
... OK (Got response in 0.46 seconds)
Writing emoji data to C:\Users\chiso\.demoji\codes.json ...
... OK


In [180]:
# load the preprocessed doc2vec data
data = pickle.load(open('eng_df_represented_doc2vec.pkl', 'rb'))
data.head()

,compound,0,1,2,3,4
0,0.0000,-0.015905,0.191896,0.311980,-0.082201,-0.054656
1,-0.9060,0.094732,-0.196510,0.232367,-0.184807,-0.250304
2,0.6249,-0.028408,0.077553,0.232172,-0.400177,0.024043
3,-0.4767,-0.025499,-0.010442,-0.289881,-0.029361,-0.232676
4,0.0000,-0.092368,0.046109,0.195366,-0.112181,-0.068188


In [188]:
# change compound from continous to categorical variable
data["compound"] = np.where(data["compound"] > 0, 1, data["compound"])
data["compound"] = np.where(data["compound"] < 0, -1, data["compound"])

In [189]:
data.head()

,compound,0,1,2,3,4
0,0.0,-0.015905,0.191896,0.311980,-0.082201,-0.054656
1,-1.0,0.094732,-0.196510,0.232367,-0.184807,-0.250304
2,1.0,-0.028408,0.077553,0.232172,-0.400177,0.024043
3,-1.0,-0.025499,-0.010442,-0.289881,-0.029361,-0.232676
4,0.0,-0.092368,0.046109,0.195366,-0.112181,-0.068188


In [190]:
# assing the X and y variables
X = data.drop("compound", axis=1)
y = data["compound"]

In [191]:
# check X
X.head()


,0,1,2,3,4
0,-0.015905,0.191896,0.311980,-0.082201,-0.054656
1,0.094732,-0.196510,0.232367,-0.184807,-0.250304
2,-0.028408,0.077553,0.232172,-0.400177,0.024043
3,-0.025499,-0.010442,-0.289881,-0.029361,-0.232676
4,-0.092368,0.046109,0.195366,-0.112181,-0.068188


In [192]:
# check y
y.head()

0    0.0
1   -1.0
2    1.0
3   -1.0
4    0.0
Name: compound, dtype: float64

In [211]:
# split  to test and train set - use 80% for training and 20% for test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42, stratify= y)

In [212]:
# train the model
clf = LogisticRegression(multi_class="multinomial", solver="lbfgs")
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       0.47      0.26      0.33      4946
         0.0       0.32      0.06      0.11      7532
         1.0       0.45      0.84      0.58      9520

    accuracy                           0.44     21998
   macro avg       0.41      0.39      0.34     21998
weighted avg       0.41      0.44      0.36     21998



In [202]:
# create a balanced data set, train and test our model on it
shuffled_df = data.sample(frac=1, random_state= 42)
print(shuffled_df["compound"].value_counts())
shuffled_df.head()


 1.0    47597
 0.0    37658
-1.0    24732
Name: compound, dtype: int64


,compound,0,1,2,3,4
39370,-1.0,-0.029322,0.249719,0.286637,0.063639,-0.072456
90544,0.0,-0.076301,0.089977,-0.088174,-0.025999,-0.029968
28550,-1.0,0.081735,0.216456,0.033149,0.000074,0.099212
86146,-1.0,0.006570,0.087801,0.419855,-0.216380,-0.064295
41622,0.0,-0.023165,0.192196,0.151058,-0.155872,0.028333


In [199]:
# put all the different compounds in a separate data set
pos = shuffled_df.loc[shuffled_df["compound"] == 1]
neu = shuffled_df.loc[shuffled_df["compound"] == 0]
neg = shuffled_df.loc[shuffled_df["compound"] == -1]

In [203]:
# balance them all to 24732, the smallest sample
neu = neu.sample(n=24732, random_state= 42)
pos = pos.sample(n=24732, random_state= 42)

In [209]:
# concatenate all dataframes
balanced_df = pd.concat([pos, neu, neg])
balanced_df.shape

(74196, 6)

In [208]:
balanced_df.head()

,compound,0,1,2,3,4
108614,1.0,0.005523,0.069174,0.066438,-0.061880,0.114920
47673,1.0,-0.060819,0.144103,0.176268,0.013342,-0.040103
98694,1.0,-0.201348,0.186996,0.256563,-0.069427,-0.008400
90420,1.0,-0.110152,0.164517,0.212028,-0.356572,-0.024280
64814,1.0,-0.016312,0.053144,0.081005,-0.140801,0.024318


In [210]:
# assing the X and y variables
X_balanced = balanced_df.drop("compound", axis=1)
y_balanced = balanced_df["compound"]

In [214]:
# split  to test and train set - use 80% for training and 20% for test
X_trainb, X_testb, y_trainb, y_testb = train_test_split(X_balanced, y_balanced, test_size= 0.2, random_state=42, stratify= y_balanced)

In [215]:
# train the model
clfb = LogisticRegression(multi_class="multinomial", solver="lbfgs")
clfb.fit(X_trainb, y_trainb)
y_predb = clf.predict(X_testb)
print(classification_report(y_testb, y_predb))

              precision    recall  f1-score   support

        -1.0       0.62      0.26      0.37      4946
         0.0       0.32      0.07      0.11      4947
         1.0       0.36      0.84      0.50      4947

    accuracy                           0.39     14840
   macro avg       0.43      0.39      0.33     14840
weighted avg       0.43      0.39      0.33     14840

